In [1]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/schiend/Desktop/DOHERTY/R-workspace/cecelia/inst')

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [2]:
base_dir = '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/s4XQza/')
im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
#im_path = os.path.join(zero_dir, 'ccidCorrected.zarr')
#im_path = os.path.join(zero_dir, 'ccidSlidingWindow.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 's4XQza')

In [3]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()
napari_utils.task_dir = task_dir

napari.manifest -> 'skan' could not be imported: Could not find file 'napari.yaml' in module 'skan'


In [4]:
channel_names = [
    'EB', 'T-A', 'T-B', 'AF', 'gen'
]

In [5]:
napari_utils.open_image(
    im_path,
    use_channel_axis = True, as_dask = True,
    downsample_z = False
)

[60, 3, 27, 512, 512]
[512, 512, 27, 3, 60]
['X', 'Y', 'Z', 'C', 'T']
[60, 3, 27, 512, 512]


In [15]:
shifts = [[-0.14,0.51,0.28],
 [-0.08,0.57,0.35],
 [-0.17,0.6, 0.12],
 [ 0.,0.84,0.02],
 [ 0.03,0.85,0.39],
 [-0.04,0.6, 0.31],
 [ 0.,1.18,0.13],
 [-0.04,0.78,0.3 ],
 [-0.11,0.8, 0.12],
 [ 0.,0.94,0.6 ],
 [-0.09,0.95,0.24],
 [ 0.03,0.46,0.31],
 [-0.03,0.62,0.17],
 [-0.04,0.65,0.32],
 [ 0.05,0.78,0.01],
 [ 0.02,0.86 -0.17],
 [-0.04,0.29 -0.07],
 [ 0.02,0.25 -0.07],
 [-0.03,0.4, 0.22],
 [-0.07,0.31,0.18],
 [ 0.,0.2, 0.32],
 [ 0.02,0.49 -0.25],
 [-0.05,0.02 -0.04],
 [ 0.04 -0.31,0.02],
 [-0.01,0.33,0.06],
 [-0.09,0.23,0.21],
 [-0.01,0.34 -0.23],
 [ 0.08,0.23 -0.04],
 [ 0.07,0.11,0.14],
 [ 0.02,0.33,0.41],
 [ 0.03,0.23 -0.09],
 [ 0.07,0.52,0.05],
 [ 0.04,0.59,0.36],
 [ 0.08,0.14,0.34],
 [ 0.15,0.53,0.22],
 [ 0.08,0.21,0.14],
 [ 0.15,0.23,0.53],
 [ 0.17 -0.26,0.25],
 [ 0.13,0.03,0.23],
 [ 0.19 -0.18,0.19],
 [ 0.04 -0.04,0.27],
 [ 0.08,0.02 -0.32],
 [ 0.03,0.26 -0.1 ],
 [ 0.01,0.31 -0.09],
 [-0.07 -0.38,0.14],
 [ 0.04,0.32,0.21],
 [-0.06,0.49 -0.04],
 [-0.13,0.28,0.16],
 [-0.04,0.62,0.31],
 [-0.01,0.91,0.47],
 [ 0.01,1.07,0.49],
 [ 0.02,0.31,0.22],
 [ 0.05,0.58,0.38],
 [-0.12,0.86,0.58],
 [-0.05,0.81,0.02],
 [-0.06,0.61,0.18],
 [ 0.,0.97,0.14],
 [ 0.07,1.04,0.01],
 [-0.07,0.68,0.14]]

In [17]:
np.array(shifts)

/var/folders/rb/m9wptf8x58x_82t7lg081kfs1fj7kv/T/ipykernel_9436/705515301.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(shifts)


array([list([-0.14, 0.51, 0.28]), list([-0.08, 0.57, 0.35]),
       list([-0.17, 0.6, 0.12]), list([0.0, 0.84, 0.02]),
       list([0.03, 0.85, 0.39]), list([-0.04, 0.6, 0.31]),
       list([0.0, 1.18, 0.13]), list([-0.04, 0.78, 0.3]),
       list([-0.11, 0.8, 0.12]), list([0.0, 0.94, 0.6]),
       list([-0.09, 0.95, 0.24]), list([0.03, 0.46, 0.31]),
       list([-0.03, 0.62, 0.17]), list([-0.04, 0.65, 0.32]),
       list([0.05, 0.78, 0.01]), list([0.02, 0.69]),
       list([-0.04, 0.21999999999999997]), list([0.02, 0.18]),
       list([-0.03, 0.4, 0.22]), list([-0.07, 0.31, 0.18]),
       list([0.0, 0.2, 0.32]), list([0.02, 0.24]), list([-0.05, -0.02]),
       list([-0.27, 0.02]), list([-0.01, 0.33, 0.06]),
       list([-0.09, 0.23, 0.21]), list([-0.01, 0.11000000000000001]),
       list([0.08, 0.19]), list([0.07, 0.11, 0.14]),
       list([0.02, 0.33, 0.41]), list([0.03, 0.14]),
       list([0.07, 0.52, 0.05]), list([0.04, 0.59, 0.36]),
       list([0.08, 0.14, 0.34]), list([0.15, 0.

In [ ]:
# get shifts summary
shifts_sum = shifts_summary(shifts, is_3D = dim_utils.is_3D())

# get new image dimensions
drift_im_shape, drift_im_shape_round = correction_im_shape(
    input_array, dim_utils, shifts_sum
)

# get first image position
first_im_pos = correction_first_im_pos(
    drift_im_shape, dim_utils, shifts_sum
)

In [10]:
napari_utils.im_data[0].shape

(60, 3, 27, 512, 512)

In [7]:
napari_utils.dim_utils.im_dim_order

['T', 'C', 'Z', 'Y', 'X']

In [9]:
napari_utils.dim_utils.dim_vals()

[60, 3, 27, 512, 512]

In [6]:
napari_utils.dim_utils.spatial_axis()

['X', 'Y', 'Z']

In [44]:
# show populations
napari_utils.show_pop_mapping('live')

(live) default/tracked


In [43]:
napari_utils.show_labels_all(
    show_labels = True,
    show_points = False,
    show_tracks = True,
    value_names = ['default']
)

{'base': '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/1/Bf68An/labels/default.zarr'}
Rendering frames...


100%|███████████████████████████████████████████| 39/39 [00:05<00:00,  7.02it/s]


In [ ]:
# save image as tiff
import py.tiff_utils as tiff_utils

tiff_utils.save_as_tiff(
    '/Volumes/USER_data/Dominik/Experiments/TEAGEN_SKIN/DATA/PuJo5p.tif',
    napari_utils.im_data[0], napari_utils.dim_utils, imagej = True)

In [ ]:
print('here')

In [ ]:
napari_utils.im_scale

In [42]:
from napari_animation import AnimationWidget
animation_widget = AnimationWidget(napari_utils.viewer)
napari_utils.viewer.window.add_dock_widget(animation_widget, area='right')

In [ ]:
pop_type = 'live'
pop_map = napari_utils.pop_utils.pop_map(napari_utils.task_dir, pop_type)
pop_data = napari_utils.pop_utils.pop_data(napari_utils.task_dir, pop_type)

In [ ]:
pop_info = pop_map['5iaWSo']

In [ ]:
pop_info